In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display
import gradio as gr

In [2]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-


In [3]:
openai = OpenAI()
claude = anthropic.Anthropic()

### GPT Model

In [4]:
system_message = "Your are assistant that answer for a given question"

def user_prompt(question):
    user_message = f"This is the question: {question}"
    return user_message

In [5]:
def gpt_chatbot_with_streaming(question):
    stream = openai.chat.completions.create(
        model = 'gpt-4.1-mini',
        messages=[
            {"role": "system", "content": system_message},
            {"role": "user", "content": user_prompt(question)}
        ],
        stream = True
    )
    
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

### Claude Model

In [6]:
def chat_bot_read_Claude(question):
    result = claude.messages.stream(
        model="claude-sonnet-4-20250514",
        max_tokens=1000,
        temperature=0.7,
        system=system_message,
        messages=[
            {"role": "user", "content": user_prompt(question)},
        ],
    )
    
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [7]:
def stream_model(prompt, model):
    if model=="GPT":
        result = gpt_chatbot_with_streaming(prompt)
    elif model=="Claude":
        result = chat_bot_read_Claude(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [8]:
view = gr.Interface(
    fn = stream_model,
    inputs = [gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Claude"], label = "Select model", value = "GPT")],
    outputs = [gr.Markdown(label = "Response:")],
    flagging_mode = "never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
